Applying https://qiskit.org/textbook/ch-algorithms/grover.html#sudoku in Quantum Counting:

In [ ]:
from utils.algorithms.counting import get_count

from qiskit import QuantumCircuit, QuantumRegister

def XOR(qc, a, b, output):
    qc.cx(a, output)
    qc.cx(b, output)
def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    # Apply transformation |s> -> |00..0> (H-gates)
    for qubit in range(nqubits):
        qc.h(qubit)
    # Apply transformation |00..0> -> |11..1> (X-gates)
    for qubit in range(nqubits):
        qc.x(qubit)
    # Do multi-controlled-Z gate
    qc.h(nqubits-1)
    qc.mct(list(range(nqubits-1)), nqubits-1)  # multi-controlled-toffoli
    qc.h(nqubits-1)
    # Apply transformation |11..1> -> |00..0>
    for qubit in range(nqubits):
        qc.x(qubit)
    # Apply transformation |00..0> -> |s>
    for qubit in range(nqubits):
        qc.h(qubit)
    # We will return the diffuser as a gate
    U_s = qc.to_gate()
    U_s.name = "U$_s$"
    return U_s

In [ ]:
clause_list = [[0,1],
               [0,2],
               [1,3],
               [2,3]]

var_qubits = QuantumRegister(4, name='v')
clause_qubits = QuantumRegister(4, name='c')
output_qubit = QuantumRegister(1, name='out')

qc = QuantumCircuit(var_qubits, clause_qubits, output_qubit)

def sudoku_oracle(qc, clause_list, clause_qubits):
    # Compute clauses
    i = 0
    for clause in clause_list:
        XOR(qc, clause[0], clause[1], clause_qubits[i])
        i += 1

    # Flip 'output' bit if all clauses are satisfied
    qc.mct(clause_qubits, output_qubit)

    # Uncompute clauses to reset clause-checking bits to 0
    i = 0
    for clause in clause_list:
        XOR(qc, clause[0], clause[1], clause_qubits[i])
        i += 1

sudoku_oracle(qc, clause_list, clause_qubits)
qc.append(diffuser(4), [0,1,2,3])
qc.draw()

var_ = QuantumRegister(4, name='v')
clause_ = QuantumRegister(4, name='c')
output_ = QuantumRegister(1, name='out')

grover_eigstate = QuantumCircuit(var_, clause_, output_)

# Initialize 'out0' in state |->
grover_eigstate.x(-1)
grover_eigstate.h(-1)

# Initialize qubits in state |s>
grover_eigstate.h(var_)

get_count(5,0.8,qc,eig_state=grover_eigstate,n=4)